In [1]:
from QNN.layers import *
import pickle

!wget -nc http://www.idi.ntnu.no/~yamanu/alexnet-hwgq.pickle
qnn = pickle.load(open("alexnet-hwgq.pickle", "rb"))

qnn

In [3]:
from imagenet_data.classes import *
len(imagenet_classes)

1000

In [40]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

# define a small utility function to first display, then prepare the
# images for classification
def prepare_gtsrb(img):
    # make sure the image is the size expected by the network
    img = img.resize((227, 227))
    display(img)
    # convert to numpy array
    img = np.asarray(img).copy().astype(np.float32)
    # we need the data layout to be (channels, rows, columns)
    # but it comes in (rows, columns, channels) format, so we
    # need to transpose the axes:
    img = img.transpose((2, 0, 1))
        
    # our network is trained with BGR instead of RGB images,
    # so we need to invert the order of channels in the channel axis:
    img = img[::-1, :, :]
    # finally, we need to subtract the mean per-channel pixel intensity
    # since this is how this network has been trained
    img[0] = img[0] - 104
    img[1] = img[1] - 117
    img[2] = img[2] - 123
    return img

# load test images and prepare them
img_grouse = prepare_gtsrb(Image.open("imagenet_data/grouse.jpg"))
display(img_grouse)
img_cat = prepare_gtsrb(Image.open("imagenet_data/cat.jpg"))
display(img_cat)
img_husky = prepare_gtsrb(Image.open("imagenet_data/husky.jpg"))
display(img_husky)

<PIL.Image.Image image mode=RGB size=227x227 at 0x7F77A9A30A70>

array([[[-30., -36., -39., ..., -43., -52., -60.],
        [-31., -36., -36., ..., -47., -53., -56.],
        [-39., -38., -35., ..., -42., -53., -48.],
        ..., 
        [-35., -20., -56., ..., -50., -27., -69.],
        [-30., -47.,  38., ..., -26.,  -5., -27.],
        [ 19., -64.,  -7., ...,  -4., -85.,   9.]],

       [[ 34.,  30.,  29., ...,  13.,  -7., -12.],
        [ 34.,  30.,  30., ...,   6., -12., -10.],
        [ 32.,  28.,  31., ...,  11.,  -9.,  -4.],
        ..., 
        [-14.,  20., -58., ...,  -3.,  32., -33.],
        [ 14., -15.,  65., ...,  18.,  35.,  13.],
        [ 39., -35.,  35., ...,  50., -46.,  34.]],

       [[ 21.,  13.,  14., ...,  -6., -31., -38.],
        [ 21.,  13.,  13., ..., -12., -35., -36.],
        [ 21.,  11.,  12., ...,  -6., -30., -25.],
        ..., 
        [  4.,  57., -12., ...,  15.,   6.,   5.],
        [ 29.,  22.,  78., ...,  37.,  42.,  44.],
        [ 49.,  -4.,  32., ...,  41., -11.,  46.]]], dtype=float32)

<PIL.Image.Image image mode=RGB size=227x227 at 0x7F77A9A07CF8>

array([[[  -2.,   10.,   20., ...,  -80.,  -78.,  -77.],
        [  -1.,   11.,   23., ...,  -81.,  -79.,  -77.],
        [   0.,   15.,   25., ...,  -77.,  -78.,  -78.],
        ..., 
        [ -90.,  -90.,  -86., ...,  -56.,  -60.,  -58.],
        [ -96.,  -92.,  -88., ...,  -53.,  -59.,  -59.],
        [ -95.,  -95.,  -90., ...,  -57.,  -55.,  -56.]],

       [[   6.,   20.,   31., ...,  -85.,  -85.,  -85.],
        [   7.,   21.,   32., ...,  -86.,  -86.,  -84.],
        [   6.,   20.,   33., ...,  -88.,  -86.,  -84.],
        ..., 
        [-110., -102.,  -87., ...,  -54.,  -57.,  -52.],
        [-109., -102.,  -92., ...,  -50.,  -51.,  -54.],
        [-108., -106.,  -94., ...,  -48.,  -47.,  -50.]],

       [[  21.,   36.,   53., ...,  -84.,  -86.,  -82.],
        [  22.,   37.,   54., ...,  -85.,  -87.,  -83.],
        [  21.,   37.,   55., ...,  -84.,  -83.,  -87.],
        ..., 
        [-119., -104.,  -81., ...,  -28.,  -27.,  -23.],
        [-115., -104.,  -89., ...,  -27., 

<PIL.Image.Image image mode=RGB size=227x227 at 0x7F77A9A30A70>

array([[[-62., -61., -55., ...,  -9., -45., -40.],
        [-61., -52., -53., ..., -68., -26.,  40.],
        [-68., -78., -71., ...,  82.,  43.,  55.],
        ..., 
        [ 17.,  -7.,  -7., ..., -26., -36.,   0.],
        [-18., -10., -29., ..., -18., -26.,   3.],
        [ 37., -34.,  66., ..., -58.,  64., -27.]],

       [[-76., -75., -69., ..., -23., -50., -52.],
        [-72., -66., -65., ..., -82., -31.,  32.],
        [-75., -85., -78., ...,  74.,  38.,  56.],
        ..., 
        [ 18.,  -5.,  -5., ..., -27., -35.,  -8.],
        [-15.,  -9., -28., ..., -19., -25.,  -8.],
        [ 44., -33.,  64., ..., -55.,  66., -34.]],

       [[-96., -95., -85., ..., -39., -67., -67.],
        [-90., -82., -80., ..., -98., -48.,  17.],
        [-86., -96., -89., ...,  59.,  25.,  44.],
        ..., 
        [  6., -15., -15., ..., -33., -45., -15.],
        [-28., -19., -38., ..., -25., -35., -14.],
        [ 33., -41.,  55., ..., -62.,  56., -41.]]], dtype=float32)

In [41]:
def imagenet_predict(img):
    # get the predictions array
    res = predict(qnn, img)
    # return the index of the largest prediction, then use the
    # classes array to map to a human-readable string
    winner_ind = np.argmax(res)
    winner_class = imagenet_classes[winner_ind]
    # the sum of the output values add up to 1 due to softmax,
    # so we can interpret them as probabilities
    winner_prob = 100 * res[winner_ind]
    print("The QNN predicts this is a %s (class %d) with %f percent probability" % (winner_class, winner_ind, winner_prob))


In [42]:
imagenet_predict(img_grouse)
imagenet_predict(img_cat)
imagenet_predict(img_husky)

The QNN predicts this is a black grouse (class 80) with 98.129130 percent probability
The QNN predicts this is a Egyptian cat (class 285) with 30.018796 percent probability
The QNN predicts this is a Eskimo dog, husky (class 248) with 59.245122 percent probability
